# 文本分类实例

## Step1 导入相关包

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

C:\Users\Administrator\.conda\envs\py310torch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step2 加载数据

In [2]:
import pandas as pd

data = pd.read_csv("./ChnSentiCorp_htl_all.csv")
data

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"
...,...,...
7761,0,尼斯酒店的几大特点：噪音大、环境差、配置低、服务效率低。如：1、隔壁歌厅的声音闹至午夜3点许...
7762,0,盐城来了很多次，第一次住盐阜宾馆，我的确很失望整个墙壁黑咕隆咚的，好像被烟熏过一样家具非常的...
7763,0,看照片觉得还挺不错的，又是4星级的，但入住以后除了后悔没有别的，房间挺大但空空的，早餐是有但...
7764,0,我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...


In [3]:
data = data.dropna()
data

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"
...,...,...
7761,0,尼斯酒店的几大特点：噪音大、环境差、配置低、服务效率低。如：1、隔壁歌厅的声音闹至午夜3点许...
7762,0,盐城来了很多次，第一次住盐阜宾馆，我的确很失望整个墙壁黑咕隆咚的，好像被烟熏过一样家具非常的...
7763,0,看照片觉得还挺不错的，又是4星级的，但入住以后除了后悔没有别的，房间挺大但空空的，早餐是有但...
7764,0,我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...


## Step3 创建Dataset

In [4]:
from torch.utils.data import Dataset

class MyDataset(Dataset):

    def __init__(self) -> None:
        super().__init__()
        self.data = pd.read_csv("./ChnSentiCorp_htl_all.csv")
        self.data = self.data.dropna()

    def __getitem__(self, index):
        return self.data.iloc[index]["review"], self.data.iloc[index]["label"]
    
    def __len__(self):
        return len(self.data)

In [5]:
dataset = MyDataset()
for i in range(5):
    print(dataset[i])

('距离川沙公路较近,但是公交指示不对,如果是"蔡陆线"的话,会非常麻烦.建议用别的路线.房间较为简单.', np.int64(1))
('商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!', np.int64(1))
('早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。', np.int64(1))
('宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小，但加上低价位因素，还是无超所值的；环境不错，就在小胡同内，安静整洁，暖气好足-_-||。。。呵还有一大优势就是从宾馆出发，步行不到十分钟就可以到梅兰芳故居等等，京味小胡同，北海距离好近呢。总之，不错。推荐给节约消费的自助游朋友~比较划算，附近特色小吃很多~', np.int64(1))
('CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风', np.int64(1))


## Step4 划分数据集

In [6]:
from torch.utils.data import random_split


trainset, validset = random_split(dataset, lengths=[0.9, 0.1])
len(trainset), len(validset)

(6989, 776)

In [7]:
for i in range(10):
    print(trainset[i])

('实在不建议大家住在那里，如果你自己没有车的话，价格贵，位置差，我第一次遇上连车都不帮你叫的酒店。竟然让客人自己提着大箱子走上20分钟去打黑车。这是我遇上的最差的酒店。如果有零分第一个打给它。', np.int64(0))
('我是九月份入住的，600多一晚，定的是普通标准简但前台主动免费升级到了15楼的海景房，态度好风景好。其实住在西环挺好的，电车和巴士方便便宜，上次我也住在附近的Ramada感觉挺好。缺点是房间有点旧、壁纸感觉不太干净。', np.int64(1))
('没想到江阴这么小的地方有这么好的酒店，真的不错，定的是豪华标间，房间很大，便于加床，没想到加的床也很舒服，卫生间很大，有浴缸和淋浴房，很干净，值得推荐。早餐也很不错。', np.int64(1))
('位置挺好，标准间特宽敞，服务水平一般．若到星海广场玩一天并晚上观看剧场真人秀可选此店入住．', np.int64(1))
('1、早餐品种比较丰富，时间跨度也可以2、房间的整体装修不错，也比较宽敞，感觉很舒服对于此酒店整体比较满意', np.int64(1))
('酒店很旧，很小，楼道里黑暗，其实就是个小旅馆的设施。建议不要去，吉林省宾馆还不错。', np.int64(0))
('先生在8/29入住时,收银台人员竟然可以将入住刷卡时的预授权800元当做消费来给客人刷信用卡,还打算撕走一张凭证,当时没注意到,但幸好三张凭证都要回来了.9/5,发现问题后,先生给酒店的收银台打了电话,在现在9/7号,都没人回复.真是素质与服务都不是一般的差!幸好另外找了信用卡银行在跟进了,真不知这800元最后会怎么样呢.补充点评2007年9月7日：在离开酒店结帐时,又说原来预授权(实际是消费)的卡用不了,要提供另一张信用卡,先生也不知就里,就用了另一张卡结帐离开了,仔细想想,这会不会是个局啊?稍不小心,800元就不知贡献给谁了?', np.int64(0))
('强烈建议携程把它剔出去。前台的服务员好像来自人猿星球，如果你不催促可以让你等上1个小时不理你。入住和结账时老夫不得不大发雷霆才算搞定。房间简陋，早餐差，夜总会的音响扰民。服务水平0分以下。十分怀疑前面的打分者有托儿。补充点评2008年4月23日：老夫是一个温和的人，这一次是第一次大发脾气。实在是忍无可忍', np.int64(0))
('就算是在上海也是非常

## Step5 创建Dataloader

In [8]:
import torch

tokenizer = AutoTokenizer.from_pretrained("hfl/rbt3")

def collate_func(batch):
    texts, labels = [], []
    for item in batch:
        texts.append(item[0])
        labels.append(item[1])
    inputs = tokenizer(texts, max_length=128, padding="max_length", truncation=True, return_tensors="pt")
    inputs["labels"] = torch.tensor(labels)
    return inputs

In [9]:
from torch.utils.data import DataLoader

trainloader = DataLoader(trainset, batch_size=32, shuffle=True, collate_fn=collate_func)
validloader = DataLoader(validset, batch_size=64, shuffle=False, collate_fn=collate_func)

In [10]:
next(enumerate(validloader))[1]

{'input_ids': tensor([[ 101, 6983, 2421,  ..., 2792, 6458,  102],
        [ 101, 6983, 2421,  ...,    0,    0,    0],
        [ 101, 2791, 7313,  ..., 2421, 1333,  102],
        ...,
        [ 101, 6821, 3221,  ...,    0,    0,    0],
        [ 101, 2791, 7313,  ..., 5445, 6241,  102],
        [ 101, 1762, 3025,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0,
        0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
        1, 1, 1

## Step6 创建模型及优化器

In [11]:
from torch.optim import Adam

model = AutoModelForSequenceClassification.from_pretrained("hfl/rbt3")

if torch.cuda.is_available():
    model = model.cuda()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/rbt3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
optimizer = Adam(model.parameters(), lr=2e-5)

## Step7 训练与验证

In [13]:
def evaluate():
    model.eval()
    acc_num = 0
    with torch.inference_mode():
        for batch in validloader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            output = model(**batch)
            pred = torch.argmax(output.logits, dim=-1)
            acc_num += (pred.long() == batch["labels"].long()).float().sum()
    return acc_num / len(validset)

def train(epoch=3, log_step=100):
    global_step = 0
    for ep in range(epoch):
        model.train()
        for batch in trainloader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            optimizer.zero_grad()
            output = model(**batch)
            output.loss.backward()
            optimizer.step()
            if global_step % log_step == 0:
                print(f"ep: {ep}, global_step: {global_step}, loss: {output.loss.item()}")
            global_step += 1
        acc = evaluate()
        print(f"ep: {ep}, acc: {acc}")

## Step8 模型训练

In [14]:
train()

ep: 0, global_step: 0, loss: 0.7103108763694763
ep: 0, global_step: 100, loss: 0.22595450282096863
ep: 0, global_step: 200, loss: 0.24033911526203156
ep: 0, acc: 0.8956185579299927
ep: 1, global_step: 300, loss: 0.1834741234779358
ep: 1, global_step: 400, loss: 0.2736663222312927
ep: 1, acc: 0.9123710989952087
ep: 2, global_step: 500, loss: 0.2908625304698944
ep: 2, global_step: 600, loss: 0.12687355279922485
ep: 2, acc: 0.9097937941551208


## Step9 模型预测

In [15]:
sen = "我觉得这家酒店不错，饭很好吃！"
id2_label = {0: "差评！", 1: "好评！"}
model.eval()
with torch.inference_mode():
    inputs = tokenizer(sen, return_tensors="pt")
    inputs = {k: v.cuda() for k, v in inputs.items()}
    logits = model(**inputs).logits
    pred = torch.argmax(logits, dim=-1)
    print(f"输入：{sen}\n模型预测结果:{id2_label.get(pred.item())}")

输入：我觉得这家酒店不错，饭很好吃！
模型预测结果:好评！


In [16]:
from transformers import pipeline

model.config.id2label = id2_label
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

Device set to use cuda:0


In [17]:
pipe(sen)

[{'label': '好评！', 'score': 0.9960254430770874}]